<a href="https://colab.research.google.com/github/divyaghodke/object_detection/blob/main/detr2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -i https://test.pypi.org/simple/ supervision==0.3.0
!pip install -q transformers
!pip install -q pytorch-lightning
!pip install -q timm
!pip install roboflow

Looking in indexes: https://test.pypi.org/simple/
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.3/812.3 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 866.2/866.2 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 60.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling chardet-5.2.0:
      Successfully uninstalled chardet-5.2.0


In [ ]:
from getpass import getpass
ROBOFLOW_API_KEY = getpass('Enter API key: ')

Enter API key: ··········


In [ ]:
!mkdir {HOME}/datasets
%cd {HOMe}/datasets


from roboflow import Roboflow
rf = Roboflow(api_key="0bdOagK5puXkdGn8wDce")
project = rf.workspace("objdetection-xqsus").project("bag-features")
version = project.version(2)
dataset = version.download("coco")

mkdir: cannot create directory ‘{HOME}/datasets’: No such file or directory
[Errno 2] No such file or directory: '{HOMe}/datasets'
/content
loading Roboflow workspace...
loading Roboflow project...


In [ ]:
dataset.location

'/content/Bag-Features-2'

In [ ]:
import os
import torchvision
from transformers import AutoFeatureExtractor

ANNOTATION_FILE_NAME = "_annotations.coco.json"
TRAIN_DIRECTORY = os.path.join(dataset.location, "train")
VAL_DIRECTORY = os.path.join(dataset.location, "valid")
TEST_DIRECTORY = os.path.join(dataset.location, "test")



image_processor = AutoFeatureExtractor.from_pretrained("facebook/detr-resnet-50")

class CocoDetection(torchvision.datasets.CocoDetection):
    def __init__(
        self,
        imgdirectory_path: str,
        image_processor,
        train: bool=True
        ):
        annotation_file_path = os.path.join(imgdirectory_path, ANNOTATION_FILE_NAME)
        super(CocoDetection, self).__init__(imgdirectory_path, annotation_file_path)
        self.image_processor = image_processor

    def __get_item__(self, idx):
      images, annotations = super(CocoDetection, self).__getitem__(idx)
      image_id = self.ids[idx]
      annotations = {'image_id': image_id, 'annotations': annotations}
      encoding = self.image_processor(images=images, annotations=annotations, return_tensors="pt")
      pixel_values = encoding['pixel_values'].squeeze()
      target = encoding['labels'][0]

      return pixel_values, target

TRAIN_DATASET = CocoDetection(image_directory_path = TRAIN_DIRECTORY, image_processor = image_processor, train=True)
VAL_DATASET = CocoDetection(image_directory_path = VAL_DIRECTORY, image_processor = image_processor, train=False)
TEST_DATASET = CocoDetection(image_directory_path = TEST_DIRECTORY, image_processor = image_processor, train=False)

print("Number of traing exampls: ", len(TRAIN_DATASET))
print("Number of validation examples: ", len(VAL_DATASET))
print("Number of test examples: ", len(TEST_DATASET))

TypeError: CocoDetection.__init__() got an unexpected keyword argument 'image_directory_path'